## Import Dependencies

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
% pip install wandb

     |████████████████████████████████| 1.8MB 8.3MB/s 
     |████████████████████████████████| 163kB 35.8MB/s 
     |████████████████████████████████| 81kB 10.8MB/s 
     |████████████████████████████████| 133kB 26.0MB/s 
     |████████████████████████████████| 102kB 11.7MB/s 
     |████████████████████████████████| 71kB 11.1MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6490 sha256=f2ea50f10766c80c7bc5b45ce9b0379b95bda0db7d6a89c623c472d7f370022b
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built subprocess32


In [4]:
# Import dependencies

from os import listdir, remove, makedirs
from os.path import join, isdir, isfile
import numpy as np
from skimage import io
from skimage.transform import resize
import random
import tensorflow as tf
from tensorflow.python.ops import variable_scope as vs
import tensorflow.keras.layers as layers
import wandb

In [6]:
wandb.login()

wandb: Currently logged in as: awesomericky (use `wandb login --relogin` to force relogin)


True

In [7]:
# Import python files

from utils import HGN_organize_data, HGN_divide_train_test
from code_HGN_senEM_train import train

## Load Preprocessed Data

In [8]:
relative_data_directory_path = '/content/drive/MyDrive/Speech2Pickup/dataset1/data_v1.2_single_channel/preprocessed4HGN_speech2pickup.npz'
    
data = np.load(relative_data_directory_path)

img_idx = data['img_idx']
sen_len = data['seq_len']
speech_inputs = data['inputs']
pos_outputs = data['outputs']
sentence = data['sentence']

print('End loading MetaData')

End loading MetaData


## Load data and divide training and test set

In [9]:
num_data = img_idx.shape[0]

img_path = '/content/drive/MyDrive/Speech2Pickup/dataset1/train_img'
script_path = '/content/drive/MyDrive/Speech2Pickup/dataset1/train_heatmap'

total_images, total_heatmaps = HGN_organize_data(img_path, script_path, model_type='speech2pickup');

args = dict()
args['img_idx'] = img_idx
args['speech_inputs'] = speech_inputs
args['pos_outputs'] = pos_outputs
args['num_data'] = num_data

idx_train, idx_test, num_train, num_test, \
train_img_idx, train_speech_inputs, train_pos_outputs, \
test_img_idx, test_speech_inputs, test_pos_outputs = \
HGN_divide_train_test(args, model_type='speech2pickup')

Loading image..
Loading 1/478
Loading 2/478
Loading 3/478
Loading 4/478
Loading 5/478
Loading 6/478
Loading 7/478
Loading 8/478
Loading 9/478
Loading 10/478
Loading 11/478
Loading 12/478
Loading 13/478
Loading 14/478
Loading 15/478
Loading 16/478
Loading 17/478
Loading 18/478
Loading 19/478
Loading 20/478
Loading 21/478
Loading 22/478
Loading 23/478
Loading 24/478
Loading 25/478
Loading 26/478
Loading 27/478
Loading 28/478
Loading 29/478
Loading 30/478
Loading 31/478
Loading 32/478
Loading 33/478
Loading 34/478
Loading 35/478
Loading 36/478
Loading 37/478
Loading 38/478
Loading 39/478
Loading 40/478
Loading 41/478
Loading 42/478
Loading 43/478
Loading 44/478
Loading 45/478
Loading 46/478
Loading 47/478
Loading 48/478
Loading 49/478
Loading 50/478
Loading 51/478
Loading 52/478
Loading 53/478
Loading 54/478
Loading 55/478
Loading 56/478
Loading 57/478
Loading 58/478
Loading 59/478
Loading 60/478
Loading 61/478
Loading 62/478
Loading 63/478
Loading 64/478
Loading 65/478
Loading 66/478
Loa

## Start Training

In [13]:
batch_size = 8
max_epoch = 300
n_mels = 40
time_steps = 303
num_hg_Depth = 4
dim_hg_feat = 256
learning_rate = 0.00001
dropout_rate = 0.0
train_model_number = 1
input_channel_type = 'single'
run_date = '2021-01-06'
encoder_args = {'num_stacks': 3, 'num_channels':[n_mels for i in range(6)], 'kernel_size':3,
                'dropout_rate': dropout_rate, 'activation': 'leaky-relu', 'return_type': 'end'}
input_shapes = (None, n_mels, time_steps)
seed = 1
training_state = True
restore_flag = 1
restore_epoch = 0
# raise ValueError('Check restore_flag and restore_epoch again!! If confident # this code')
"""
1) restore_flag == 0
- Restore encoder model weights only
- restore_epoch == 0

2) restore_flag == 1
- Restore total model weights
- restore_epoch != 0
"""
if restore_flag == 0 and restore_epoch != 0:
    raise ValueError('Check restore_flag and restore_epoch')
elif restore_flag == 1 and restore_epoch == 0:
    raise ValueError('Check restore_flag and restore_epoch')

wandb_config = {'batch_size': batch_size,
                'max_epoch': max_epoch,
                'n_mels': n_mels,
                'num_hg_Depth': num_hg_Depth,
                'dim_hg_feat': dim_hg_feat,
                'learning_rate': learning_rate,
                'dropout_rate': dropout_rate,
                'train_model_number': train_model_number,
                'input_channel_type': input_channel_type,
                'seed': seed,
                'training_state': training_state,
                'num_train': num_train,
                'num_test': num_test,
                'date': run_date}

if restore_flag == 2:
    id = wandb.util.generate_id()
    print('Wandb id: {}'.format(id))
    with open('/content/drive/MyDrive/Speech2Pickup/dataset1_model/HGN_senEM_model_wo/wandbID.txt', 'w') as f:
        f.write(str(id))
elif restore_flag == 1:
    id = '9k5xv50p'
else:
    raise ValueError('Unsupported restore_flag')

wandb_run = wandb.init(project='Speech2Pickup', name='HGN_senEM', id=id, resume="allow", config=wandb_config)

train(img_resize = 256, # must be fixed
      heatmap_resize = 64, # must be fixed
      num_hg_Depth = num_hg_Depth, # good be fixed
      dim_hg_feat = dim_hg_feat, # good be fixed
      n_mels = n_mels,
      time_steps = time_steps,
      encoder_model_path = '/content/drive/MyDrive/Speech2Pickup/sentenceEM_model/SOTA/encoder_model/sentenceEM_encoder_model',
      encoder_args = encoder_args,
      input_shapes = input_shapes,
      seed = seed,
      training_state = training_state,
      batch_size = batch_size, 
      max_epoch = max_epoch,
      num_train = num_train,
      save_stride = 1,
      learning_rate = learning_rate, # good be fixed
      dropout_rate = dropout_rate,
      restore_flag = restore_flag,
      restore_path = '/content/drive/MyDrive/Speech2Pickup/dataset1_model/HGN_senEM_model_wo/model.ckpt',
      restore_epoch = restore_epoch,
      total_images = total_images,
      total_heatmaps = total_heatmaps,
      train_speech_inputs = train_speech_inputs,
      train_img_idx = train_img_idx,
      train_pos_outputs = train_pos_outputs,
      model_save_path = '/content/drive/MyDrive/Speech2Pickup/dataset1_model/HGN_senEM_model_wo/model.ckpt')

Wandb id: 9k5xv50p




Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.wher

KeyboardInterrupt: ignored

In [ ]:
# Reset tensor graph
# Finish wandb process

tf.reset_default_graph()
wandb.finish()

In [ ]:
# Check variable list (debugging)

variables_names = [v.name for v in tf.trainable_variables()]
variables_names